In [129]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from scipy.misc import imresize
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import random
# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [148]:
image_size = 500  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.
image_size_resized = 160
exampleImage = np.ndarray(shape=(image_size,image_size,3),dtype=np.float32)

class BatchGenerator(object):
    folder = ""
    globalIndex = 0
    def __init__(self, folder):
          self.folder = folder
    def getCategory(self,imageName):
        animal = imageName.split(".")[0]
        if animal == "cat":
            return 0
        else:
            return 1
    def nextBatch(self,shuffle=True,batchImageNumber=500):
          folder = self.folder
          print("loading data...")
          """Load the data for a single letter label."""
          image_files = os.listdir(folder)
          datasetInput = np.zeros(shape=(batchImageNumber, image_size_resized, image_size_resized,3),
                                 dtype=np.float32)
          dataSetLabel = np.zeros(shape=(batchImageNumber),
                                 dtype=int)
          num_images = 0
          if shuffle == True:
            random.shuffle(image_files)
          
          trucks = len(image_files)//batchImageNumber
          index = self.globalIndex%(trucks-1)
          imageCount = 0
          for i in range(index*batchImageNumber,(index+1)*batchImageNumber):
            image = image_files[i]
            image_file = os.path.join(folder, image)
            try:
              image_data = (ndimage.imread(image_file).astype(float) - 
                            pixel_depth / 2) / pixel_depth
              fullResule_resized = []
              if(image_data.shape[0]*image_data.shape[1]>image_size*image_size):
                fullResule_resized = imresize(image_data, (image_size_resized,image_size_resized), interp='bilinear', mode=None)
              else:
                fullResult = np.full(exampleImage.shape,0, dtype=exampleImage.dtype)
                #padding image with 0s
                fullResult[:image_data.shape[0],:image_data.shape[1],:image_data.shape[2]] = image_data
                fullResule_resized = imresize(fullResult, (image_size_resized,image_size_resized), interp='bilinear', mode=None)
              datasetInput[imageCount] = fullResule_resized
              dataSetLabel[imageCount] = getCategory(image)
              #print(dataSetLabel[imageCount])
              # show image if want
              #plt.imshow(fullResule_resized)
            except IOError as e:
              print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
            imageCount += 1
          self.globalIndex += 1
          #dataSetLabel = dataSetLabel.reshape(-1)
          return datasetInput,dataSetLabel




In [147]:
dataSetInput,datasetLabel = load_image_from_pickle("cat_dog_train")
batchGenerator = BatchGenerator("cat_dog_train")
datasetInput,dataSetLabel = batchGenerator.nextBatch()


loading data...
[1 0 1 0 0 1 1 0 1 1 0 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 1 1 1 0 0 1 0 0
 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1
 1 1 0 0 0 0 0 1 1 1 1 0 0 1 1 0 1 0 1 1 1 0 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0
 1 1 1 0 0 0 1 1 1 0 0 0 1 0 0 1 1 1 1 0 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0
 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0
 1 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 1 0
 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 1 1 0 1 1 1
 0 0 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0
 1 1 0 1 1 1 0 0 1 1 1 0 1 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1
 1 0 1 1 1 1 0 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 0 0 1 0 0
 1 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 1 1 1 1 0 0 1 0 1 0 1 1 0 1 1 0 1 1 1
 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0 1 1 0 1 0 1 1 1
 0 1 1 1 

In [149]:

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD

depth = 16
num_hidden = 300
num_channels = 3
patch_size = 5
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.96,nesterov=True)  


model = Sequential()
model.add(Convolution2D(depth, patch_size, patch_size, border_mode='same',subsample=(1,1) ,input_shape=(image_size_resized, image_size_resized, num_channels)))
model.add(Activation('relu')) #16x28x28x16
model.add(MaxPooling2D(pool_size=(4, 4)))#16x14x14x16
model.add(Convolution2D(depth, patch_size, patch_size, border_mode='same'))#16x14x14x16
model.add(MaxPooling2D(pool_size=(4, 4)))#16x7x7x16
model.add(Flatten())#16x(7x7x16)
model.add(Activation('relu'))
model.add(Dense(num_hidden))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
optimizer="adam",metrics=["accuracy"])

for i in range(10):
    print(i," :batch")
    train_dataset, train_labels = batchGenerator.nextBatch()
    model.fit(train_dataset, train_labels, nb_epoch=1,verbose=1)




loading data...
0  :batch
loading data...
Epoch 1/1
500/500 [==============================] - 10s - loss: 8.4176 - acc: 0.4720    
1  :batch
loading data...
Epoch 1/1
500/500 [==============================] - 9s - loss: 7.4292 - acc: 0.5340     
2  :batch
loading data...
Epoch 1/1
500/500 [==============================] - 8s - loss: 7.4929 - acc: 0.5300     
3  :batch
loading data...
Epoch 1/1
500/500 [==============================] - 9s - loss: 7.7480 - acc: 0.5140     
4  :batch
loading data...
Epoch 1/1
500/500 [==============================] - 9s - loss: 7.5567 - acc: 0.5260     
5  :batch
loading data...
Epoch 1/1
500/500 [==============================] - 9s - loss: 7.9393 - acc: 0.5020     
6  :batch
loading data...
Epoch 1/1
500/500 [==============================] - 9s - loss: 8.3219 - acc: 0.4780     
7  :batch
loading data...
Epoch 1/1
500/500 [==============================] - 10s - loss: 7.8755 - acc: 0.5060    
8  :batch
loading data...
Epoch 1/1
500/500 [===========